In [1]:
!pip cache purge
!pip install numpy pandas matplotlib scipy statsmodels scikit-learn tensorflow wordcloud

# spaCyとginzaをアンインストール
!pip uninstall -y spacy ginza ja_ginza

# 必要なバージョンを再インストール
!pip install spacy==3.0
!pip install ginza
!pip install --no-cache-dir spacy ginza ja_ginza

!pip install chardet

Files removed: 155
Found existing installation: spacy 3.8.2
Uninstalling spacy-3.8.2:
  Successfully uninstalled spacy-3.8.2
Found existing installation: ginza 5.2.0
Uninstalling ginza-5.2.0:
  Successfully uninstalled ginza-5.2.0
Found existing installation: ja-ginza 5.2.0
Uninstalling ja-ginza-5.2.0:
  Successfully uninstalled ja-ginza-5.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 47.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [167 lines of output]
      
      Error compiling Cython file:
      ------------------------------------------------------------
      ...
          int length
      
      
      cdef class Vocab:
          cdef Pool mem
          cpdef readonly StringStore strings
                ^
      ------------------------------------------------------------
  

In [2]:
#q1-2
import csv
import spacy
from concurrent.futures import ProcessPoolExecutor

# giNZAのロード（パイプラインの一部を無効化して高速化）
nlp = spacy.load("ja_ginza", disable=["parser", "ner"])

# ファイルパスを指定
file_path = "../data/practice3-1/testset1m.txt"

# 名詞を格納するセット（重複を除くためセットを使用）
unique_nouns = set()

# タイトルから名詞を抽出する関数
def extract_nouns(title):
    doc = nlp(title)
    return {token.text for token in doc if token.pos_ == "NOUN"}

# ファイルを読み込み、並列でスレッドタイトルを抽出
def process_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        reader = csv.reader(file, delimiter="\t")  # タブ区切りで読み込み
        titles = [row[2].strip() for row in reader if len(row) > 2]  # スレッドタイトルを抽出
    return titles

# 並列処理を使って名詞を抽出
def parallel_extract(file_path):
    # ファイルから並列でスレッドタイトルを抽出
    titles = process_file(file_path) # 自分で定義した関数

    # プロセスプールを使って並列処理
    with ProcessPoolExecutor() as executor:
        # titles各要素から名詞を抽出
        results = executor.map(extract_nouns, titles) # titlesリストの要素に対して並列にextract_nounsを実行

    # 各プロセスで得られた名詞をユニークに統合
    for result in results:
        unique_nouns.update(result)

# 並列処理で名詞抽出を実行
parallel_extract(file_path)

# ユニーク名詞の件数
unique_noun_count = len(unique_nouns)

# 結果を表示
print(f"ユニーク名詞数: {unique_noun_count}")

ユニーク名詞数: 114807


In [1]:
import csv
from collections import Counter, defaultdict
import spacy
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor
from itertools import islice

# GiNZAのロード
nlp = spacy.load("ja_ginza", disable=["parser", "ner"])

file_path = "../data/practice3-1/testset1m.txt"

# 一括処理の関数（nlp.pipeを使用）
def process_titles_chunk(titles):
    keywords = []
    for doc in nlp.pipe(titles, batch_size=1000, n_process=-1):  # バッチ処理
        nouns = [token.text for token in doc if token.pos_ == "NOUN"]
        keywords.extend(nouns)
    return keywords

# ストリーム処理用関数
def read_file_in_chunks(file_path, chunk_size=10000):
    with open(file_path, "r", encoding="utf-8") as file:
        reader = csv.reader(file, delimiter="\t")
        while True:
            chunk = list(islice(reader, chunk_size))
            if not chunk:
                break
            yield chunk

# 月ごとのキーワードを処理する関数
def process_monthly_keywords(args):
    month_key, titles = args
    keywords = process_titles_chunk(titles)
    keyword_counts = Counter(keywords)
    return month_key, keyword_counts.most_common(10)

# メイン処理
def main(file_path):
    monthly_titles = defaultdict(list)

    # ストリーム処理でファイルを読み込む
    for chunk in read_file_in_chunks(file_path, chunk_size=10000):
        for row in chunk:
            if len(row) > 2:
                title = row[2].strip()  # スレッドタイトル
                date_str = row[5].strip()  # 投稿日時
                try:
                    post_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
                    month_key = post_date.strftime("%Y-%m")
                    monthly_titles[month_key].append(title)
                except ValueError:
                    # 無効な日時データをスキップ
                    continue

    # 並列処理を実行
    with ProcessPoolExecutor() as executor:
        results = executor.map(process_monthly_keywords, monthly_titles.items())

    # 結果を辞書にまとめる
    top_keywords_per_month = dict(results)

    # 結果を表示
    for month, top_keywords in sorted(top_keywords_per_month.items()):
        print(f"{month}:")
        for keyword, count in top_keywords:
            print(f"  {keyword}: {count}")

# 実行
if __name__ == "__main__":
    main(file_path)


/opt/conda/lib/python3.11/site-packages/spacy/language.py:1737: UserWarning: [W127] Not all `Language.pipe` worker processes completed successfully
  warnings.warn(Warnings.W127)
/opt/conda/lib/python3.11/site-packages/spacy/language.py:1737: UserWarning: [W127] Not all `Language.pipe` worker processes completed successfully
  warnings.warn(Warnings.W127)
/opt/conda/lib/python3.11/site-packages/spacy/language.py:1737: UserWarning: [W127] Not all `Language.pipe` worker processes completed successfully
  warnings.warn(Warnings.W127)
/opt/conda/lib/python3.11/site-packages/spacy/language.py:1737: UserWarning: [W127] Not all `Language.pipe` worker processes completed successfully
  warnings.warn(Warnings.W127)
/opt/conda/lib/python3.11/site-packages/spacy/language.py:1737: UserWarning: [W127] Not all `Language.pipe` worker processes completed successfully
  warnings.warn(Warnings.W127)
/opt/conda/lib/python3.11/site-packages/spacy/language.py:1737: UserWarning: [W127] Not all `Language.pip

KeyboardInterrupt: 

In [ ]:
#q1-4

import csv
from collections import defaultdict, Counter
import spacy
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor

nlp = spacy.load("ja_ginza", disable=["parser", "ner"])

file_path = "../data/practice3-1/testset1m.txt"

monthly_keywords = defaultdict(list)  # 月ごとのキーワードリスト

# ファイルを読み込み、月別にキーワードを抽出
with open(file_path, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")  # タブ区切りで読み込み
    for row in reader:
        if len(row) > 2:  # スレッドタイトルと日時が存在するかチェック
            title = row[2].strip()  # スレッドタイトルを取得
            date_str = row[5].strip()  # 投稿日時を取得

            try:
                # 日時をdatetimeに変換して月を取得
                post_date = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
                month_key = post_date.strftime("%Y-%m")  # 年月キー（例: "2020-02")

                # GiNZAを用いて名詞を抽出
                doc = nlp(title)
                nouns = [token.text for token in doc if token.pos_ == "NOUN"]

                # 月ごとのキーワードリストに追加
                monthly_keywords[month_key].extend(nouns)
            except ValueError:
                # 無効な日付データをスキップ
                continue

# 全体の頻度を集計
all_keywords = []  # 全体のキーワードリスト
for keywords in monthly_keywords.values():  # values()でキーワードのみ取得
    all_keywords.extend(keywords)

total_keyword_counts = Counter(all_keywords)  # 全体の頻度を保持する辞書

# 特徴語を抽出する関数
def extract_characteristic_keywords(args):
    month, keywords, total_keyword_counts = args
    keyword_counts = Counter(keywords)  # 各月のキーワード頻度({"猫":2}のように格納)
    characteristics = {}
    for word, count in keyword_counts.items():
        overall_ratio = count / total_keyword_counts[word]  # 全体頻度に対する割合
        if 0.5 <= overall_ratio < 1.0:  # 全体頻度の 50% 以上を占めるが、100% ではない
            characteristics[word] = overall_ratio
    # 上位10件を抽出
    sorted_characteristics = sorted(characteristics.items(), key=lambda x: x[1], reverse=True)
    return month, sorted_characteristics[:10]

# 並列処理で特徴語を抽出
with ProcessPoolExecutor() as executor:
    results = executor.map(
        extract_characteristic_keywords,  # 特徴語を抽出
        [(month, keywords, total_keyword_counts) for month, keywords in monthly_keywords.items()]
    )

# 結果を辞書に格納
top_keywords_per_month = dict(results)

# 結果を表示
for month, top_keywords in sorted(top_keywords_per_month.items()):
    print(f"{month}:")
    for keyword, ratio in top_keywords:
        print(f"  {keyword}: {ratio:.2f}")
